In [33]:
import pandas as pd

user_table = pd.read_csv('user_table.csv')
home_page = pd.read_csv('home_page_table.csv')
search_page = pd.read_csv('search_page_table.csv')
payment_page = pd.read_csv('payment_page_table.csv')
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')

In [34]:
# Меняем название одинаковой колонки в каждом датасете
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})


In [35]:
# Склеиваем последовательно все датасеты с помощью метода merge по идентификатору user_id
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [36]:
# Для удоства последующих расчетов, перекодируем названия событий в 0/1 с помощью лямбда-функции
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)

df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [37]:
# Использовать датасет с текущего урока и построить сегментированную воронку конверсии пользователей интернет-магазина по полу.

dfg = df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().unstack('sex').unstack('sex').reset_index()
dfg.columns.name = None
dfg = dfg.rename(columns = {'index':'Page'})
dfg.head()

,Page,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [65]:
# Создать 2 новые фичи на основе колонки “date”: месяц и день недели
import datetime

df['date_year'] = pd.DatetimeIndex(df['date']).year
df['date_month'] = pd.DatetimeIndex(df['date']).month
df['date_week'] = pd.DatetimeIndex(df['date']).weekday
df['day_name']=pd.DatetimeIndex(df['date']).day_name()
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,date_year,date_month,date_week,day_name
0,450007,2015-02-28,Desktop,Female,1,0,0,0,2015,2,5,Saturday
1,756838,2015-01-13,Desktop,Male,1,0,0,0,2015,1,1,Tuesday
2,568983,2015-04-09,Desktop,Male,1,1,0,0,2015,4,3,Thursday
3,190794,2015-02-18,Desktop,Female,1,1,0,0,2015,2,2,Wednesday
4,537909,2015-01-15,Desktop,Male,1,0,0,0,2015,1,3,Thursday


In [59]:
# Определить самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count).

top_month=df.groupby('date_month').agg({'payment_confirmation':'sum'})
top_month.head

<bound method NDFrame.head of             payment_confirmation
date_month                      
1                            189
2                            173
3                             44
4                             46>

In [66]:
top_month=df.groupby('day_name').agg({'payment_confirmation':'sum'})
top_month.head

<bound method NDFrame.head of            payment_confirmation
day_name                       
Friday                       54
Monday                       77
Saturday                     70
Sunday                       62
Thursday                     66
Tuesday                      69
Wednesday                    54>

In [67]:
# Выяснить, в какой день недели лучше покупают женщины, 
# а в какой мужчины с помощью одного из методов построения сводных таблиц 
# (groupby, pivot_table или crosstab).
pd.pivot_table(df, index = 'sex', 
               columns = 'day_name', 
               values = 'payment_confirmation', 
               aggfunc = 'sum', 
               margins=True)

# Вывод: мужчины лучше всего покупают по четвергам, а женщины - по понедельникам.


day_name,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,All
sex,,,,,,,,
Female,26,43,36,35,31,40,30,241
Male,28,34,34,27,35,29,24,211
All,54,77,70,62,66,69,54,452
